# Text Sentiment Analysis of IMDB Movie reviews using NLP (Word2Vec and RNN) for MYM Intern Assesment

## Assesment Objectives: 
### - Preprocessing the data
### - Converting Text(words) to Vectors using word2vec 
### - Using the word representations given by word2vec to feed a RNN and training the model
### - Testing the model


In [24]:
!pip freeze | grep gensim ##Checking the version of Gensim - Word2Vec

gensim==4.2.0


## The Data

### Starting with 20% of the sentences from TensorFlow Datasets of IMDB reviews to check the RAM compatibility of the PC to train the model faster by splitting the datasets as X_train, y_train, X_test and y_test.
### Then preprocessing the textual data to create input features for a natural language processing (NLP) model.



In [78]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
      assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
      len_train = int(percentage_of_sentences/100*len(train_sentences))
      train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
      len_test = int(percentage_of_sentences/100*len(test_sentences))
      test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=20)

In [79]:
y_test

array([1, 1, 0, ..., 1, 0, 0])

## First, training a word2vec model (with the arguments that we want) on your training sentence. Store it into the `word2vec` variable.

In [80]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=10, window=10)

In [82]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

### It's a good practice to check check the following for `X_train_pad` and `X_test_pad`:
#### - they are numpy arrays
#### - they are 3-dimensional
#### - the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`\\
#### - the first dimension is of the size of your `X_train` and `X_test`

In [83]:
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

## Baseline Model

In [84]:
# It is always good to have a very simple model to test your own model against
# Baseline accuracy can be to predict the label that is the most present in `y_train`.
from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

y_pred = 0 if counts[0] > counts[1] else 1

print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))

Number of labels in train set {0: 2474, 1: 2526}
Baseline accuracy:  0.499


## The Model

In [85]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

# writing a RNN model with Masking, LSTM and Dense layers.

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', #compiling the model with rmsprop optimizer
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

model = init_model()

In [86]:
#  Fiting the model on embedded and padded data with the early stopping criterion.

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
110/110 [==============================] - 7s 37ms/step - loss: 0.6849 - accuracy: 0.5554 - val_loss: 0.6843 - val_accuracy: 0.5653
Epoch 2/100
110/110 [==============================] - 3s 32ms/step - loss: 0.6465 - accuracy: 0.6311 - val_loss: 0.6471 - val_accuracy: 0.6273
Epoch 3/100
110/110 [==============================] - 4s 32ms/step - loss: 0.5882 - accuracy: 0.6954 - val_loss: 0.5988 - val_accuracy: 0.6853
Epoch 4/100
110/110 [==============================] - 3s 32ms/step - loss: 0.5454 - accuracy: 0.7280 - val_loss: 0.5911 - val_accuracy: 0.6800
Epoch 5/100
110/110 [==============================] - 3s 32ms/step - loss: 0.5241 - accuracy: 0.7483 - val_loss: 0.6195 - val_accuracy: 0.6840
Epoch 6/100
110/110 [==============================] - 4s 32ms/step - loss: 0.5016 - accuracy: 0.7586 - val_loss: 0.5485 - val_accuracy: 0.7227
Epoch 7/100
110/110 [==============================] - 4s 33ms/step - loss: 0.4818 - accuracy: 0.7731 - val_loss: 0.5478 - val_accuracy:

In [87]:
# Evaluating the model on the test set.

result = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {result[1]*100:.3f}%')

The accuracy evaluated on the test set is of 76.280%


## Trained Word2Vec - Transfer Learning


### The accuracy of the above the baseline model, might be quite low. By improving the quality of the embedding we can Improve accuracy of the model.

### Let's improve the quality of our embedding, instead of just loading a larger corpus, let's benefit from the embedding that others have learned. Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.

### Listing all the different models available in the word2vec using gensim api.

In [88]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [89]:
#Let's load one of the pre-trained word2vec embedding spaces. 

word2vec_transfer = api.load("glove-wiki-gigaword-100")

In [91]:
print(len(word2vec_transfer.key_to_index))
print(len(word2vec_transfer['dog']))

400000
100


In [92]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [93]:
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [96]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()

model.fit(X_train_pad_2, y_train, 
          batch_size = 32,
          epochs=30,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/30
110/110 [==============================] - 7s 41ms/step - loss: 0.6766 - accuracy: 0.5774 - val_loss: 0.6399 - val_accuracy: 0.6513
Epoch 2/30
110/110 [==============================] - 4s 34ms/step - loss: 0.6259 - accuracy: 0.6606 - val_loss: 0.6478 - val_accuracy: 0.6253
Epoch 3/30
110/110 [==============================] - 4s 35ms/step - loss: 0.5878 - accuracy: 0.6994 - val_loss: 0.6211 - val_accuracy: 0.6567
Epoch 4/30
110/110 [==============================] - 4s 35ms/step - loss: 0.5421 - accuracy: 0.7366 - val_loss: 0.5511 - val_accuracy: 0.7327
Epoch 5/30
110/110 [==============================] - 4s 34ms/step - loss: 0.5047 - accuracy: 0.7589 - val_loss: 0.5931 - val_accuracy: 0.6993
Epoch 6/30
110/110 [==============================] - 4s 35ms/step - loss: 0.4745 - accuracy: 0.7803 - val_loss: 0.5050 - val_accuracy: 0.7647
Epoch 7/30
110/110 [==============================] - 4s 34ms/step - loss: 0.4507 - accuracy: 0.7983 - val_loss: 0.4906 - val_accuracy: 0.7687

In [97]:
result = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {result[1]*100:.3f}%')

The accuracy evaluated on the test set is of 80.580%


### There is a significant improvement in the accuracy after Transfer learning.

In [99]:
!git add Sentiment_Analysis_using_NLP.ipynb

In [101]:
!git commit -m 'Sentiment Analysis using NLP'

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	my_model/
	saved_model.pb
	~/

nothing added to commit but untracked files present (use "git add" to track)


In [102]:
!git push -u origin master

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.28 KiB | 2.28 MiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:pavankumarhm/Sentiment-Analysis-Assesment.git
   febec41..2242c44  master -> master
branch 'master' set up to track 'origin/master'.
